In [91]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torchvision.transforms as Trans

import torch.nn.init as init


In [92]:
BATCH_SIZE = 100
NUM_WORKERS = 4

EPOCHS = 8

In [93]:
transform = Trans.ToTensor()


train_dataset = datasets.CIFAR10(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.CIFAR10(
    root='datasets',
    train=False,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

Files already downloaded and verified
Files already downloaded and verified


In [94]:
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 12, 5, 1, 2),
            # nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.Conv2d(12, 32, 3, 2, 1),
            # nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1),
            # nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        x = self.net(x)
        return x

CELoss = nn.CrossEntropyLoss()
net = MyNet()
net.cuda()

optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [95]:
def train(net, optimizer, criterion):
    running_loss = 0
    for ind, (images, labels) in enumerate(pbar := tqdm(train_dataloader)):
        images = images.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()

        output = net(images)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss
        pbar.set_description(f"loss: {loss}")
    with torch.no_grad():
        train_loss = running_loss / len(train_dataloader)
    return train_loss


def valid(net, criterion):
    running_loss = 0
    correct_total = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.cuda()
            labels = labels.cuda()
            output = net(images)

            loss = criterion(output, labels)
            running_loss += loss

            pred = output.max(dim = 1, keepdim=True)[1]
            correct_total += pred.eq(labels.view_as(pred)).sum()
            
        precison = correct_total / len(test_dataloader.dataset)
        valid_loss = running_loss / len(test_dataloader)
        return valid_loss, precison

In [96]:
for epoch in (pbar := tqdm(range(EPOCHS))):
    train_loss = train(net, optimizer, CELoss)
    valid_loss, prec = valid(net, CELoss)

    print(f"[{epoch}] train/valid loss: {train_loss:.4f}/{valid_loss:.4f} acc: {prec:.4f}")
    pbar.set_description(f"train/valid loss: {train_loss:.4f}/{valid_loss:.4f} acc: {prec:.4f}")

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

[0] train/valid loss: 2.0073/1.7106 acc: 0.3846


  0%|          | 0/500 [00:00<?, ?it/s]

[1] train/valid loss: 1.6122/1.5040 acc: 0.4638


...

[5] train/valid loss: 1.0095/1.1298 acc: 0.5925


  0%|          | 0/500 [00:00<?, ?it/s]

[6] train/valid loss: 0.8526/1.0962 acc: 0.6127


  0%|          | 0/500 [00:00<?, ?it/s]

[7] train/valid loss: 0.6981/1.1924 acc: 0.6098
